In [3]:
import json
from bs4 import BeautifulSoup

The purpose of this notebook is to get all my beautifulSoup calls correctly formatted

In [68]:
path = '../chic_data/chic_export_24.json'
is_exists = []
check = []
features_collect = []
htmls = []

In [69]:
with open(path) as json_data:
    for line in json_data:
        entry = json.loads(line)
        html = entry['html'] 
        id_num = entry['id']
        htmls.append([html, id_num])
        


In [70]:
len(htmls)



24

In [71]:
sample1 = htmls[6][0]
sample2 = htmls[12][0]
sample3 = htmls[18][0]
sample4 = htmls[23][0]

In [72]:
soup = BeautifulSoup(sample1, "html.parser")

In [73]:
soup

<!DOCTYPE html>\n\n<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:wb="http://open.weibo.com/wb">\n<head prefix="og: http://ogp.me/ns# app_chictopia: http://ogp.me/ns/apps/app_chictopia#">\n<meta charset="unicode-escape"/>\n<title>Men's Chai Jackets | "Fly me Fly me!!" by techin | Chictopia</title>\n<meta content="Discover this look wearing Chai Jackets tagged art, fall fashion trends, fashion, happy, photography - Fly me Fly me!! by techin styled for Casual, Clubbing in the Spring" name="description"/>\n<link href="http://www.chictopia.com/assets/application-8f996d59e9a18e50a6a5732bd45bc0fe.css" media="all" rel="stylesheet" type="text/css"/>\n<script src="//ajax.googleapis.com/ajax/libs/jquery/1.8.0/jquery.min.js" type="text/javascript"></script>\n<script src="https://d1b7u2ujuyo838.cloudfront.net/images/libraries/jquery.placeholder.js"></script>\n<script src="http://www.chictopia.com/assets/application-620b76c66db2

In [74]:
htmls[6][1]

100019

In [16]:
def check_status(html):
    '''some entries either refer to urls with no photo or return an error
    this function will alert to pages that do not exist and flag pages that should
    rechecked'''
    soup = BeautifulSoup(html, "html.parser")
    output = ''
    text = str(soup.find())
    if text.startswith('<response [200]') or text.startswith('<response [404]'):
        output = 'check'
    elif soup.title.text == u"The page you are looking for doesn't exist | Chictopia":
        output = "DNE"
    else:
        output = "passed"
    return output

In [79]:
post_id = soup.find("input", {"id" :"photo_id"})
if post_id is not None:
    post_id = int(post_id.get("value"))
    fd['post_id'] = post_id
else:
    fd['post_id'] = 0


fd['post_id']


100019

In [60]:
title = soup.find("h1", { "class":"photo_title"})
if title is not None:
    title = title.text
    fd['title'] = title.encode('ascii', 'ignore')
else:
    fd['title'] = 'No Title'
    
fd['title']

'Fly me Fly me!!'

In [63]:
date = soup.find("meta", {"itemprop":"dateCreated"})
if date is not None:
    date = date.get("content")
    fd['date'] = date.encode('ascii', 'ignore')
else:
    fd['date'] = "No Date"
        
fd['date']

'2009-04-20'

In [64]:
sub_photos = soup.find("div", {"class":"subphoto_items"})
if sub_photos is not None:
    sub_photos = sub_photos.findChildren("img")
    sub_photos = [photo.get("src") for photo in sub_photos]
    fd['subphotos'] = sub_photos
else:
    fd['subphotos'] = "No Subphotos"

fd['subphotos']

[u'http://images2.chictopia.com/photos/techin/7043981989/7043981989_sm.jpg',
 u'http://images2.chictopia.com/photos/techin/7043981989/5136844237/5136844237_sm.jpg',
 u'http://images2.chictopia.com/photos/techin/7043981989/5388070635/5388070635_sm.jpg',
 u'http://images2.chictopia.com/photos/techin/7043981989/5308872942/5308872942_sm.jpg']

In [65]:
main_photo = soup.find("div", {"id" : "image_wrap"})
if main_photo is not None:
    main_photo = main_photo.findChild("img").get("src")
    fd['main_photo'] = main_photo.encode("ascii", "ignore")
else:
    fd['main_photo'] = "No Main Photo"

fd['main_photo']

'http://images0.chictopia.com/photos/techin/7043981989/7043981989_400.jpg'

In [80]:
vote_tag = "vote_text_{}".format(post_id)

votes = soup.find("div", {"id": vote_tag})
if votes is not None:
    votes = votes.text
    votes = int(votes.split()[0])
    fd['votes'] = int(votes)
else:
    fd['votes'] = 0

    
print votes
print fd['votes']


found <div class="left action_unclicked_show cursor" data-actiontext="VOTE!" data-counttext="11 votes" id="vote_text_100019" onclick="loginPrompt('vote');clickTrack(&quot;photo_vote&quot;);" onmouseout="user_action_button_mouseout('vote', '100019')" onmouseover="user_action_button_mouseover('vote','100019')" style="width:97px;margin-left:3px;">
                11 votes
              </div> for votes field
11
11


In [82]:
comments_tag = "comment_text_{}".format(post_id)
comments = soup.find("div", {"id": comments_tag})
if comments is not None:
    comments = comments.text
    comments = int(comments.split()[0])
    fd['comments'] = int(comments)
else:
    fd['comments'] = 0
    
print fd['comments']

11


In [84]:
faves_tag = "favorite_text_{}".format(post_id)
faves = soup.find("div", {"id": faves_tag})
if faves is not None:
    faves = faves.text
    faves = int(faves.split()[0])
    fd['favorites'] = int(faves)
else:
    fd['favorites'] = 0
    
fd['favorites']



1

In [24]:
tags = soup.find("div", {"id": "tag_boxes"}).findChildren("a")
tags = [tag.text for tag in tags]
tags


[u'Casual', u'Cocktail', u'fall']

In [25]:
desc = soup.find("div", {"id": "photo_description"}).text
desc

u'\nscarf and belt vintage.\ndress topshop.\nsilver sling bag topshop. '

In [26]:
#this doesn't exist on this page. I probably need a contingency for any time an element doesn't exist

links = soup.find("div",{"class": "garmentLinks"})
links

In [27]:
style_council = soup.find("div", {"class":"help"}).findChild("img").get("alt")
style_council

u'Style_council'

In [87]:
#need contingency for when people have 0 followers like here
followers = soup.find("div", {"id":"fan_count"})
if followers is not None:
    followers = followers.text
    if followers != '\n\n':
        fd['followers'] = int(followers)
else:
    fd['followers'] = 0
    
fd['followers']    

19

In [37]:
soup.find("div", {"id":"follow"}).text

u'\n\n'

In [ ]:
username = soup.find("div",{ "id":"name_div"}).findChild("a").text
username

In [ ]:
location = soup.find("div", {"id":"loc_div"}).findChild("a").text
location

In [ ]:
chic_points = soup.find("div", {"itemprop": "author"}).findChildren("div", {"class":"px10"})[2].text
chic_points = chic_points.split()[0]
chic_points

In [89]:
chic_points = soup.find("div", {"itemprop": "author"}).findChildren("div", {"class":"px10"})
chic_points

[<div class="left cap bold px10 bg_lg" id="fan_count" style="line-height:25px;height:25px;width:92px;text-align:center">\n        19\n      </div>,
 <div class="av_info px10 " id="loc_div" style="line-height:16px;">\n          from <a alt="Bangkok, Thailand" href="/Bangkok-Street-Style/info" style="text-transform:capitalize">Bangkok, Thailand</a>\n</div>,
 <div class="left px10 " style="line-height:16px;">888 chic points</div>]

In [ ]:
date = soup.find("meta", {"itemprop":"dateCreated"}).get("content")
date

In [ ]:
#create a features dictionary
fd = {}

In [ ]:
labels = [username, title, date, location, desc, chic_points, followers, votes, comments, faves, \
              style_council, links, tags, main_photo, sub_photos]



In [ ]:
def extract_feats(html):
    '''
    '''

    soup = BeautifulSoup(html, "html.parser")
    #features dictionary
    fd = {}
    
    #find post id for reference
    post_id = soup.find("input", {"id" :"photo_id"})
    if post_id is not None:
        post_id = int(post_id.get("value"))
        fd['post_id'] = post_id
    else:
        fd['post_id'] = 0

    #find title of post
    title = soup.find("h1", { "class":"photo_title"})
    if title is not None:
        title = title.text
        fd['title'] = title.encode('ascii', 'ignore')
    else:
        fd['title'] = 'No Title'

    #find date of post
    date = soup.find("meta", {"itemprop":"dateCreated"})
    if date is not None:
        date = date.get("content")
        fd['date'] = date.encode('ascii', 'ignore')
    else:
        fd['date'] = "No Date"

    #find smaller photos url
    sub_photos = soup.find("div", {"class":"subphoto_items"})
    if sub_photos is not None:
        sub_photos = sub_photos.findChildren("img")
        sub_photos = [photo.get("src") for photo in sub_photos]
        fd['subphotos'] = sub_photos
    else:
        fd['subphotos'] = "No Subphotos"

    #find main photo
    main_photo = soup.find("div", {"id" : "image_wrap"})
    if main_photo is not None:
        main_photo = main_photo.findChild("img").get("src")
        fd['main_photo'] = main_photo.encode("ascii", "ignore")
    else:
        fd['main_photo'] = "No Main Photo"

    #number of votes for this post
    vote_tag = "vote_text_{}".format(post_id)
    votes = soup.find("div", {"id": vote_tag})
    if votes is not None:
        votes = votes.text
        votes = int(votes.split()[0])
        fd['votes'] = int(votes)
    else:
        fd['votes'] = 0

    #number of comments for this post
    comments_tag = "comment_text_{}".format(post_id)
    comments = soup.find("div", {"id": comments_tag})
    if comments is not None:
        comments = comments.text
        comments = int(comments.split()[0])
        fd['comments'] = int(comments)
    else:
        fd['comments'] = 0

    #number of favorites for this post
    faves_tag = "favorite_text_{}".format(post_id)
    faves = soup.find("div", {"id": faves_tag})
    if faves is not None:
        faves = faves.text
        faves = int(faves.split()[0])
        fd['favorites'] = int(faves)
    else:
        fd['favorites'] = 0

    #find all tags
    tags = soup.find("div", {"id": "tag_boxes"})
    if tags is not None:
        tags = tags.findChildren("a")
        tags = [tag.text.encode('ascii', 'ignore') for tag in tags]
        fd['tags'] = tags
    else:
        fd['tags'] = "No Tags"

    #find description
    desc = soup.find("div", {"id": "photo_description"})
    if desc is not None:
        desc = desc.text
        fd['photo_desc'] = desc.encode('ascii', 'ignore')
    else:
        fd['photo_desc'] = "No Description"

    #find links to garments
    links = soup.find("div",{"class": "garmentLinks"})
    if links is not None:
        links = links.findChildren("a")
        fd['garment_links'] = [link.text.encode('ascii', 'ignore') for link in links]
    else:
        fd['garment_links'] = None

    #find styleCouncl status
    style_council = soup.find("div", {"class":"help"})
    if style_council is not None:
        style_council = style_council.findChild("img").get("alt")
        fd['style_council'] = style_council.encode('ascii', 'ignore')
    else:
        fd['style_council'] = "Not Style Council (or..no badge?)"

    #find number of followers
    followers = soup.find("div", {"id":"fan_count"})
    if followers is not None:
        followers = followers.text
        if followers != '\n\n':
            fd['followers'] = int(followers)
    else:
        fd['followers'] = 0

    fd['followers']  

    #find username
    username = soup.find("div",{ "id":"name_div"})
    if username is not None:
        username = username.findChild("a").text
        fd['username'] = username.encode('ascii', 'ignore')
    else:
        fd['username'] = "No Username"


    #find location
    location = soup.find("div", {"id":"loc_div"})
    if location is not None:
        location = location.findChild("a")
        if location is not None:
            location = location.text
            fd['location'] = location.encode('ascii', 'ignore')
    else:
        fd['location'] = "n/a"

    #find num chic points
    chic_points = soup.find("div", {"itemprop": "author"})
    if chic_points is not None:
        chic_points = chic_points.findChildren("div", {"class":"px10"})[2].text
        chic_points = chic_points.split()[0]
        fd['chic_points'] = int(chic_points)
    else:
        fd['chic_points'] = 0

    ''''''
    return fd
